In [12]:
import speech_recognition as sr
from gtts import gTTS
from groq import Groq
import os
from dotenv import load_dotenv
load_dotenv()
def transcribe_audio(max_tries=3):
    r=sr.Recognizer()
    for attempt in range(max_tries):
        with sr.Microphone() as source:
            r.adjust_for_ambient_noise(source, duration=2)
            audio=r.listen(source, phrase_time_limit=5)
            try:
                return r.recognize_google(audio,language="en-US")
            except sr.UnknownValueError:
                print("Could not understand audio")
    return ""
GROQ_API_KEY=os.getenv("FELLOWSHIP_GROQ_KEY")

history = [{"role": "system", "content": "You are a helpful assistant."}]
def generate_response(text):
    global history
    history.append({"role":"user","content":text})
    try:
        client=Groq(api_key=GROQ_API_KEY)
      
        chat_completion=client.chat.completions.create(
              messages=[history[0]]+history[-5:],
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        max_tokens=50
        )
        response = chat_completion.choices[0].message.content
        history.append({"role":"assistant","content":response})

        return response
    except Exception as e:
        print("GROQ ERROR",e)
        return ""
    
def text_to_speech(text,output_file="output.mp3"):
    tts=gTTS(text=text,lang="en")
    tts.save(output_file)
    print("Audio saved successfully.")
if __name__=="__main__":
    print("Assistant is running... Say something! (say 'stop' to end)")
    counter=1
    while True:
        text=transcribe_audio()
        if text.strip()=="":
            print("Didn't catch that, pleae try again")
            continue
        elif text.lower() in ["stop","exit","quit"]:
            print("Conversation ended.")
            break
    
        response=generate_response(text)
        text_to_speech(response, f"response_{counter}.mp3")
        counter+=1

Assistant is running... Say something! (say 'stop' to end)


Audio saved successfully.
Could not understand audio
Audio saved successfully.
Conversation ended.
